In [1]:
import MetaTrader5 as mt5
from datetime import datetime
import pandas as pd
import time 
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import warnings

warnings.filterwarnings('ignore')

In [25]:

def black_scholes_price(S, K, T, r, sigma, option_type):
    """Calcula preço da opção pelo modelo Black-Scholes"""
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'Call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:  # Put
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def implied_volatility(market_price, S, K, T, r, option_type):
    """Calcula volatilidade implícita usando preço de mercado (Last, Bid, Ask)"""
    if T <= 0 or S <= 0 or K <= 0 or market_price <= 0:
        return np.nan
    try:
        func = lambda sigma: black_scholes_price(S, K, T, r, sigma, option_type) - market_price
        return brentq(func, 1e-6, 3.0)  # tenta encontrar sigma entre 0.000001 e 3.0
    except (ValueError, RuntimeError):
        return np.nan

def calcular_gregas_blackscholes_com_iv(df,vol = 0.14, taxa_r=0.10):
    df = df.copy()
    df['Data'] = pd.to_datetime(df['Data'], errors='coerce')
    df['Vencimento'] = pd.to_datetime(df['Vencimento'], errors='coerce')

    df = df.dropna(subset=['Data', 'Vencimento', 'Cotacao_Ativo', 'Strike'])
    df['Dias_para_Venc'] = (df['Vencimento'] - df['Data']).dt.days
    df['T'] = df['Dias_para_Venc'] / 252

    df = df[(df['T'] > 0) & (df['Cotacao_Ativo'] > 0) & (df['Strike'] > 0)]

    S = df['Cotacao_Ativo'].astype(float)
    K = df['Strike'].astype(float)
    T = df['T'].astype(float)
    r = taxa_r
    tipo = df['Tipo']

    # Calcula volatilidade implícita com base em Last, Bid, Ask
    df['IV_Last'] = [
        implied_volatility(p, s, k, t, r, o)
        for p, s, k, t, o in zip(df['Last'], S, K, T, tipo)
    ]
    df['IV_Bid'] = [
        implied_volatility(p, s, k, t, r, o)
        for p, s, k, t, o in zip(df['Bid'], S, K, T, tipo)
    ]
    df['IV_Ask'] = [
        implied_volatility(p, s, k, t, r, o)
        for p, s, k, t, o in zip(df['Ask'], S, K, T, tipo)
    ]

    # Use IV_Last como referência para gregas e preço teórico
    sigma_est = vol
    d1_est = (np.log(S / K) + (r + 0.5 * sigma_est ** 2) * T) / (sigma_est * np.sqrt(T))
    d2_est = d1_est - sigma_est * np.sqrt(T)

    # Use IV_Last como referência para gregas e preço teórico
    sigma = df['IV_Last']
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    # Valor justo
    df['BS_Preco_IV'] = np.where(
        df['Tipo'] == 'Call',
        S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2),
        K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    )
    # Valor justo
    df['BS_Preco_Est'] = np.where(
        df['Tipo'] == 'Call',
        S * norm.cdf(d1_est) - K * np.exp(-r * T) * norm.cdf(d2_est),
        K * np.exp(-r * T) * norm.cdf(-d2_est) - S * norm.cdf(-d1_est)
    )
    # Gregas
    df['Delta'] = np.where(
        df['Tipo'] == 'Call',
        norm.cdf(d1),
        -norm.cdf(-d1)
    )
    df['Gamma'] = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    df['Vega'] = S * norm.pdf(d1) * np.sqrt(T) 
    df['Theta'] = np.where(
        df['Tipo'] == 'Call',
        (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)) / 252,
        (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 252
    )
    df['Rho'] = np.where(
        df['Tipo'] == 'Call',
        K * T * np.exp(-r * T) * norm.cdf(d2) / 100,
        -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100
    )

    return df


In [26]:
# String inicial
base_string = 'BOVA'

# Gerar a lista com as variações de 'VALEA' até 'VALEZ'
result = [base_string + chr(i) for i in range(ord('A'), ord('Z') + 1)]

# Exibir o resultado
print(result[23])

BOVAX


In [27]:
mt5.initialize()

mt5.login(12712431,'Hammstein08*', 'Rico-PRD')
lista_opcoes = []


for nome in result:
    ticker = mt5.symbols_get(nome)
    # print(len(ticker))
    for j in range(1,len(ticker)):
        lista_opcoes.append(ticker[j].name)

In [28]:
for i in range(len(lista_opcoes)):
    ticker = lista_opcoes[i]
    
    # Obter as informações do símbolo de uma vez
    mt5.symbol_select(ticker,True)

time.sleep(10)
    

In [29]:
dicionario_opcoes = {}
for i in range(len(lista_opcoes)):
    ticker = lista_opcoes[i]
    
    # Obter as informações do símbolo de uma vez
    
    symbol_info = mt5.symbol_info(ticker)
    ativo_base = mt5.symbol_info(ticker).basis
    mt5.symbol_select(ativo_base,True)
    if (ativo_base == ''):
        cota = 0
    else:
        cota = mt5.symbol_info(ativo_base).last


    if symbol_info is not None:
        # Construir o valor do dicionário com as informações
        dicionario_opcoes[ticker] = {
            'Last': symbol_info.last,
            'Bid': symbol_info.bid,
            'Ask': symbol_info.ask,
            'Volume': symbol_info.session_volume,
            'Data': datetime.fromtimestamp(symbol_info.time).strftime('%Y-%m-%d'),
            'Vencimento': datetime.fromtimestamp(symbol_info.expiration_time).strftime('%Y-%m-%d'),
            'Strike': symbol_info.option_strike,
            'Cotacao_Ativo' : cota,
            'Tipo': symbol_info.option_right,
            # 'Vencimento': symbol_info.expiration_time,
            'Abertura': symbol_info.session_open
        }

# Exibir o dicionário para conferir
print(dicionario_opcoes)

grade_opcoes = pd.DataFrame(dicionario_opcoes).T

opcoes_negociadas = grade_opcoes[grade_opcoes['Volume'] != 0]
opcoes_negociadas.loc[opcoes_negociadas['Tipo'] == 0, 'Tipo'] = 'Call'
opcoes_negociadas.loc[opcoes_negociadas['Tipo'] == 1, 'Tipo'] = 'Put'

df_opcoes = opcoes_negociadas.sort_values(by='Vencimento')

{'BOVAA100': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '1969-12-31', 'Vencimento': '2026-01-16', 'Strike': 10.0, 'Cotacao_Ativo': 134.42, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA119': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '2025-07-09', 'Vencimento': '2027-01-15', 'Strike': 119.0, 'Cotacao_Ativo': 134.42, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA125': {'Last': 21.87, 'Bid': 10.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '2025-07-09', 'Vencimento': '2026-01-16', 'Strike': 125.0, 'Cotacao_Ativo': 134.42, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA126': {'Last': 0.0, 'Bid': 0.01, 'Ask': 0.0, 'Volume': 0.0, 'Data': '2025-07-09', 'Vencimento': '2026-01-16', 'Strike': 126.0, 'Cotacao_Ativo': 134.42, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA127': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '1969-12-31', 'Vencimento': '2026-01-16', 'Strike': 127.0, 'Cotacao_Ativo': 134.42, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA128': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, '

In [30]:
mt5.symbol_select(ativo_base)

dados = mt5.copy_rates_from_pos(ativo_base, mt5.TIMEFRAME_D1, 0, 180)

# 4. Converte para DataFrame e ajusta datas
df = pd.DataFrame(dados)
df['time'] = pd.to_datetime(df['time'], unit='s')
print(df[['time', 'open', 'high', 'low', 'close', 'tick_volume']])
vol = ((df['close'].pct_change().rolling(21).std())*np.sqrt(252)).iloc[-1]

          time    open    high     low   close  tick_volume
0   2024-10-15  127.50  127.80  126.75  127.40        74336
1   2024-10-16  127.73  128.65  127.27  128.07        81708
2   2024-10-17  126.60  127.43  126.40  127.31        69809
3   2024-10-18  128.12  128.23  126.66  127.00       111994
4   2024-10-21  127.36  127.62  126.66  126.75       134049
..         ...     ...     ...     ...     ...          ...
175 2025-07-03  136.45  138.15  136.09  137.92        77130
176 2025-07-04  137.39  138.45  137.39  138.13        29900
177 2025-07-07  138.05  138.09  136.17  136.45        72051
178 2025-07-08  136.34  136.40  135.66  136.15        39229
179 2025-07-09  135.94  135.94  134.38  134.43        47138

[180 rows x 6 columns]


In [31]:
a = calcular_gregas_blackscholes_com_iv(df_opcoes)

a[['Last','Cotacao_Ativo','Strike','T','IV_Last']]

,Last,Cotacao_Ativo,Strike,T,IV_Last
BOVAS141W2,5.97,134.42,141.0,0.007937,NaN
BOVAG144W2,0.01,134.42,144.0,0.007937,0.315593
BOVAG142W2,0.01,134.42,142.0,0.007937,0.258120
BOVAG141W2,0.01,134.42,141.0,0.007937,0.228505
BOVAG140W2,0.02,134.42,140.0,0.007937,0.218718
...,...,...,...,...,...
BOVAX145,7.38,134.42,145.0,2.095238,0.196664
BOVAX13,4.55,134.42,130.0,2.091270,0.214492
BOVAX139,6.69,134.42,140.0,2.091270,0.208719
BOVAP175,16.8,134.42,175.0,2.563492,0.189762


In [37]:
df_comp = calcular_gregas_blackscholes_com_iv(df_opcoes, vol = vol,taxa_r=0.13390)
df_comp['UpSide'] = df_comp['BS_Preco_Est'] - df_comp['Last'] 
df_comp['Vol_Hist'] = vol
df_comp['Diff'] = abs(df_comp['Cotacao_Ativo'] - df_comp['Strike'])
df_comp = df_comp[['Last', 'Bid', 'Ask', 'Volume', 'Vencimento', 'Strike','Diff', 'Tipo', 'Dias_para_Venc', 'IV_Last', 'IV_Bid', 'IV_Ask', 'Vol_Hist', 'BS_Preco_Est','UpSide', 'Delta', 'Gamma', 'Vega', 'Theta', 'Rho']]



df_comp.loc[(df_comp['Dias_para_Venc'] >=15) & (df_comp['Dias_para_Venc'] <= 45) ].sort_values('Diff')

,Last,Bid,Ask,Volume,Vencimento,Strike,Diff,Tipo,Dias_para_Venc,IV_Last,IV_Bid,IV_Ask,Vol_Hist,BS_Preco_Est,UpSide,Delta,Gamma,Vega,Theta,Rho
BOVAT134W1,1.09,1.09,0.0,20100.0,2025-08-01,134.0,0.42,Put,23,0.120970,0.120970,NaN,0.140131,1.375416,0.285416,-0.330583,0.073772,14.717068,-0.014512,-0.041552
BOVAG134W4,2.47,2.44,2.51,28904.0,2025-07-25,134.0,0.42,Call,16,0.117024,0.114606,0.120236,0.140131,2.760855,0.290855,0.658798,0.092566,12.427287,-0.091188,0.054658
BOVAS134W4,1.21,1.18,1.23,27099.0,2025-07-25,134.0,0.42,Put,16,0.140408,0.138048,0.141979,0.140131,1.206472,-0.003528,-0.364506,0.079001,12.725402,-0.029158,-0.031877
BOVAH134W4,5.39,1.18,0.0,50.0,2025-08-22,134.0,0.42,Call,44,0.151133,NaN,NaN,0.140131,5.168862,-0.221138,0.674128,0.042445,20.237904,-0.080042,0.148808
BOVAH34,3.86,3.85,3.87,542159.0,2025-08-15,134.0,0.42,Call,37,0.097941,0.097348,0.098532,0.140131,4.613543,0.753543,0.734347,0.065010,16.891802,-0.072756,0.139265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BOVAH70,66.92,64.32,66.15,10.0,2025-08-15,70.0,64.42,Call,37,1.147107,NaN,0.921589,0.140131,65.782755,-1.137245,0.959848,0.001463,4.452219,-0.102014,0.091183
BOVAH500,86.7,83.99,85.86,1000.0,2025-08-15,50.0,84.42,Call,37,1.692207,NaN,1.386922,0.140131,85.393396,-1.306604,0.969925,0.000782,3.511841,-0.103516,0.064129
BOVAT220,80.76,81.52,83.36,20.0,2025-08-15,220.0,85.58,Put,37,NaN,0.579157,0.879071,0.140131,81.297057,0.537057,NaN,NaN,NaN,NaN,NaN
BOVAH320,103.2,101.63,103.53,23.0,2025-08-15,32.07,102.35,Call,37,1.751869,NaN,2.018843,0.140131,102.974336,-0.225664,0.993786,0.000194,0.903429,-0.037533,0.044612


          time    open    high     low   close  tick_volume
0   2024-10-15  127.50  127.80  126.75  127.40        74336
1   2024-10-16  127.73  128.65  127.27  128.07        81708
2   2024-10-17  126.60  127.43  126.40  127.31        69809
3   2024-10-18  128.12  128.23  126.66  127.00       111994
4   2024-10-21  127.36  127.62  126.66  126.75       134049
..         ...     ...     ...     ...     ...          ...
175 2025-07-03  136.45  138.15  136.09  137.92        77130
176 2025-07-04  137.39  138.45  137.39  138.13        29900
177 2025-07-07  138.05  138.09  136.17  136.45        72051
178 2025-07-08  136.34  136.40  135.66  136.15        39229
179 2025-07-09  135.94  135.94  134.38  134.52        46189

[180 rows x 6 columns]


In [13]:
# LIMPA A MARKWT WATCH DO MT5

lista_remocao = list(grade_opcoes.index)

for nome in lista_remocao:
    mt5.symbol_select(nome,False)

mt5.symbol_select(ativo_base,False)

    

True

In [14]:

# mt5.shutdown()
